In [0]:
CREATE OR REPLACE FUNCTION shm.3w.latest_n_obs(
  well_number_param BIGINT DEFAULT -1 COMMENT "Well number between 1 and 50, must be an integer",
  n_obs_param INT DEFAULT 5 COMMENT "Number of observations to return")
RETURNS TABLE (
  well_number DOUBLE, 
  timestamp TIMESTAMP,
  `T-JUS-CKP` DOUBLE,
  `T-TPT` DOUBLE,
  `P-TPT` DOUBLE,
  `P-MON-CKP` DOUBLE,
  `P-PDG` DOUBLE,
  `QGL` DOUBLE
  )
COMMENT "Gives the latest n observations for the well. If well_number_param is -1, returns the most recent n observations for all wells."
RETURN
SELECT well_number, timestamp, `T-JUS-CKP`, `T-TPT`, `P-TPT`, `P-MON-CKP`, `P-PDG`, `QGL`
FROM (
  SELECT 
    well_number,
    timestamp,
    `T-JUS-CKP`,
    `T-TPT`,
    `P-TPT`,
    `P-MON-CKP`,
    `P-PDG`,
    `QGL`,
    ROW_NUMBER() OVER (PARTITION BY well_number ORDER BY timestamp DESC) as row_num
  FROM shm.3w.well_data
  WHERE (well_number_param = -1 OR well_number = well_number_param)
  ORDER BY timestamp DESC
  LIMIT 1000 -- Use a large constant limit
) subquery
WHERE row_num <= n_obs_param